In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

%matplotlib inline
#from expo import expo

In [2]:
from prep import prep

In [3]:
### Test the kwargs
# initialize a default dictionary 
# iterate through kwargs and replace or append. 
# test, 
# insert into the function
# test it again

In [4]:
df = pd.read_csv('test_file_by_2.csv')

In [5]:
df.head()

,date,county,category,total_amount_sold,total_sales_dollars
0,2016-09-01,LINN,american vodkas,32541,276725.76
1,2016-10-01,LINN,american vodkas,35255,282506.41
2,2016-11-01,LINN,american vodkas,39521,327080.42
3,2016-12-01,LINN,american vodkas,38355,311999.82
4,2017-01-01,LINN,american vodkas,32319,268929.67


In [6]:
f_list = ['county','category']

In [7]:
df_prep = prep(df,'date','total_amount_sold',f_list).df_ready


this is inside methdod, after serilazie function  Index(['county', 'category', 'date', 'srl_num', 'index', 'total_amount_sold'], dtype='object')


/workspaces/exponential_smoothing/prep.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stage_df = stage_df.append(temp2,ignore_index=True)
/workspaces/exponential_smoothing/prep.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stage_df = stage_df.append(temp2,ignore_index=True)


In [8]:
df_prep.head()


,county,category,date,srl_num,total_amount_sold
0,LINN,canadian whiskies,2016-01-01,1,13349.0
1,LINN,canadian whiskies,2016-02-01,1,21764.0
2,LINN,canadian whiskies,2016-03-01,1,13223.0
3,LINN,canadian whiskies,2016-04-01,1,12492.0
4,LINN,canadian whiskies,2016-05-01,1,19208.0


In [9]:
temp_df = df_prep[df_prep.srl_num == '1']

In [10]:
temp_df.head()

,county,category,date,srl_num,total_amount_sold
0,LINN,canadian whiskies,2016-01-01,1,13349.0
1,LINN,canadian whiskies,2016-02-01,1,21764.0
2,LINN,canadian whiskies,2016-03-01,1,13223.0
3,LINN,canadian whiskies,2016-04-01,1,12492.0
4,LINN,canadian whiskies,2016-05-01,1,19208.0


In [11]:
# transform dataframe
#temp_df.columns
temp_df = temp_df[['date','total_amount_sold']]



In [12]:
temp_df.set_index('date',inplace=True)


In [13]:
temp_df.head()

,total_amount_sold
date,
2016-01-01,13349.0
2016-02-01,21764.0
2016-03-01,13223.0
2016-04-01,12492.0
2016-05-01,19208.0


In [14]:
horizon = 6

In [15]:
#def_param = {'seasonal_periods': 12, 'trend': 'add', 'seasonal': 'add', 'use_boxcox': True, 'initialization_method': 'estimated'}
def_param = {'seasonal_periods': 52,  'trend': 'add', 'seasonal': 'add', 'use_boxcox': True, 'initialization_method': 'heuristic'}
print(def_param.keys())


dict_keys(['seasonal_periods', 'trend', 'seasonal', 'use_boxcox', 'initialization_method'])


In [16]:
def test_func(x, **kwargs):
    def_param = {'seasonal_periods': 12, 'trend': 'add', 'seasonal': 'add', 'use_boxcox': True, 'initialization_method': 'estimated'}
    if kwargs:
        for key, value in kwargs.items():
            def_param[key] = value
    else:
        print('using default parameters')

    return def_param

In [17]:
kwargs_test = {'seasonal_periods': 12,'initialization_method':'estimated','missing':'raise'}


In [18]:
a_test = test_func(2)  

using default parameters


In [19]:
a_test

{'seasonal_periods': 12,
 'trend': 'add',
 'seasonal': 'add',
 'use_boxcox': True,
 'initialization_method': 'estimated'}

In [20]:
b_test = test_func(2,**kwargs_test)


In [21]:
b_test

{'seasonal_periods': 12,
 'trend': 'add',
 'seasonal': 'add',
 'use_boxcox': True,
 'initialization_method': 'estimated',
 'missing': 'raise'}

In [22]:
# fit1 = ExponentialSmoothing(temp_df,seasonal_periods = 12,
#     trend='add',seasonal='add',
#     use_boxcox=True,
#     initialization_method='estimated')

In [23]:
fit2 = ExponentialSmoothing(temp_df,**b_test,).fit()

/workspaces/exponential_smoothing/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [24]:
fcast = fit2.forecast(horizon)


In [25]:
type(fcast)

pandas.core.series.Series

In [26]:
periods = {365:'D',7:'D',12:'M',52:'W',4:'Q'}
seasonal_periods = 12

In [27]:
periods[12]

'M'

In [28]:
fc_index = pd.date_range(start=temp_df.index.max(), periods=horizon+1, freq=periods[12])[1:]

In [29]:
ts_id = '1'

In [30]:
[ts_id]*horizon

['1', '1', '1', '1', '1', '1']

In [31]:
fcast.values

array([19712.61259664, 26477.41727576, 21408.22334195, 24689.35073723,
       18491.67461571, 22772.7004094 ])

In [32]:
output = {'date':fc_index,'forecast':fcast.values,'ts_id':[ts_id]*horizon}
#output = {'forecast':fcast,'ts_id':[ts_id]*horizon}
output_df = pd.DataFrame(output)


In [33]:
output_df.head()

,date,forecast,ts_id
0,2021-09-30,19712.612597,1
1,2021-10-31,26477.417276,1
2,2021-11-30,21408.223342,1
3,2021-12-31,24689.350737,1
4,2022-01-31,18491.674616,1


In [34]:
output_df.reset_index()

,index,date,forecast,ts_id
0,0,2021-09-30,19712.612597,1
1,1,2021-10-31,26477.417276,1
2,2,2021-11-30,21408.223342,1
3,3,2021-12-31,24689.350737,1
4,4,2022-01-31,18491.674616,1
5,5,2022-02-28,22772.700409,1
